In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk
import math
from nltk import pos_tag, word_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
data = pd.read_csv("winemag-data-130k-v2.csv")
data.drop(columns=["Unnamed: 0"],inplace=True)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Filippo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Filippo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Filippo\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
data = data.drop_duplicates(["description","price","points","taster_name","title"])
len(data)

119988

In [3]:
"""
missingOnlyName = 0
missingOnlyTwitter = 0
missingBoth = 0
lis = []
for idx,row in data.iterrows():
    #print(row["taster_name"])
    if(pd.isnull(row["taster_name"]) and (not pd.isnull(row["taster_twitter_handle"]))):
        missingOnlyName = missingOnlyName + 1
    if(pd.isnull(row["taster_twitter_handle"]) and (not pd.isnull(row["taster_name"]))):
        missingOnlyTwitter = missingOnlyTwitter + 1
    if(pd.isnull(row["taster_name"]) and pd.isnull(row["taster_twitter_handle"])):
        missingBoth = missingBoth + 1
    couple = [row["taster_name"],row["taster_twitter_handle"]]
    if not couple in lis:
        lis.append(couple)
print(missingOnlyName,missingOnlyTwitter,missingBoth)
print(lis,len(lis))
"""
data.drop(["taster_twitter_handle"],axis=1,inplace=True)

In [4]:
avgPriceWinery = data.groupby("winery")#.price.mean()
#avgPriceWinery.get_group("1+1=3").price.mean()
data['price'] = data.apply(
    lambda row: avgPriceWinery.get_group(row["winery"]).price.mean() if pd.isnull(row['price']) else row['price'],
    axis=1
)
data["price"].fillna(data["price"].mean(),inplace=True)

#prende solo recensioni con autore conosciuto, ha senso come cosa?
#data.dropna(subset=["taster_name"],inplace=True)

data.fillna("unknown",inplace=True)

In [5]:
actualData = data.groupby("country").filter(lambda x: len(x)>600) 
len(actualData)

117652

In [6]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,country,description,designation,points,price,province,region_1,region_2,taster_name,title,variety,winery,word_count
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,17.0,Sicily & Sardinia,Etna,unknown,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,24
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,unknown,unknown,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,39
2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,28
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,unknown,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,33
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,41


In [7]:
#desc = actualData.iloc[0]["description"]
#tokens = word_tokenize(desc)
#tagged = pos_tag(tokens)
#tagged

In [8]:
#tagged[0][1]

In [9]:
taggedDescriptions = []
for idx, row in actualData.iterrows():
    desc = row["description"]
    tokens = word_tokenize(desc)
    tagged = pos_tag(tokens)
    taggedDescriptions.append(tagged)

In [10]:
descriptions = []
for item in taggedDescriptions:
    processed = ""
    for el in item:
        if el[1].startswith("N") or el[1].startswith("J"):  #se iniziano con N o J sono nomi o aggettivi, quello che ci serve di più
            processed = processed + " " + el[0]
    descriptions.append(processed)

In [11]:
descriptions[3]

' Pineapple rind lemon pith orange blossom aromas palate opulent notes honey-drizzled guava mango way astringent semidry finish'

In [12]:
actualData["description"] = descriptions

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
#lower case everything
actualData['description'] = actualData['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
actualData.to_csv("dataAfterPOS.csv", encoding='utf-8', index=False)

In [16]:
#rimosso tutto il removibile? come viene adesso la situa? 
actualData["description"]

0         aromas tropical fruit broom brimstone herb pal...
1         ripe fruity wine firm tannins juicy red berry ...
2         tart snappy flavors lime flesh dominate green ...
3         pineapple rind lemon pith orange blossom aroma...
4         much regular bottling rough tannic rustic eart...
5         blackberry typical navarran whiff green herbs ...
6         bright informal aromas candied berry white pep...
7         dry restrained wine offers profusion acidity f...
8         savory thyme notes accent sunnier flavors pres...
9         great depth flavor fresh apple pear fruits tou...
10        soft supple plum oaky structure cabernet % mer...
11        dry wine spicy tight taut texture mineral char...
12        wine chalky tannic backbone otherwise juicy ex...
13        oak oak-driven aromas coffee bean espresso coc...
14        years generations tradition winery leaner styl...
15        zesty orange peels apple notes mineral-toned r...
16        baked plum molasses balsamic v

In [ ]:
#nelle descrizioni si trovano anche i nomi di altri vini, tipo "ah ha 30% di bordeaux". Rimuovere nomi dei vini o lasciare tutto?

In [17]:
actualData["word_count"] = actualData["description"].apply(lambda x: len(str(x).split(" ")))
actualData.head()
print(actualData["word_count"].mean())

19.969690273008535


E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
